# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [1]:
import json

In [2]:
with open("courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [ ]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [4]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])

In [ ]:
print('notice that the instructor and code are tuples now')
courses[1246]

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to asnwer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

In [ ]:
## a: how many faculty taught COSI courses last year?
len([i for i in courses if i['subject'] == "COSI" ])

In [ ]:
## b: what is the total number of students taking COSI courses last year?
def filterItemByGroup(subject, item, group = 'subject'):
    return [i[item] for i in courses if i[group] == subject]
sum(countStudentBySubject("COSI", "enrolled"))

In [ ]:
## c: what was the median size of a COSI course last year (counting only those courses with at least 10 students)
from numpy import median
median([i for i in filterItemByGroup("COSI", "enrolled") if i >= 10])

In [ ]:
## d: create a list of tuples (E,S) where S is a subject and E is the number of students enrolled in courses in that subject, sort it and print the top 10. This shows the top 10 subjects in terms of number of students taught.
subjects = set([course['subject'] for course in courses])
## this function sort the input tuple by counts and output the top n rows where n is assigned by the second parameter
def topNFromTuple(tupleIn, n = 10):
    subTuples_sorted = sorted(subTuples, key = lambda x : x[1], reverse = True)
    return subTuples_sorted[0:n]

subTuples = [(i, sum(filterItemByGroup(i, "enrolled"))) for i in subjects]
topTenFromTuple(subTuples)

In [ ]:
## e: do the same as in (d) but print the top 10 subjects in terms of number of courses offered
subTuples = [(i, len(set(filterItemByGroup(i, "code")))) for i in subjects] # use set to remove duplicates in a list of "code"s
topNFromTuple(subTuples)

In [ ]:
## f: do the same as (d) but print the top 10 subjects in terms of number of faculty teaching courses in that subject
subTuples = [(i, len(set(filterItemByGroup(i, "instructor")))) for i in subjects] # set to remove duplicates in a list of instructors
topNFromTuple(subTuples)

In [ ]:
## g: list the top 20 faculty in terms of number of students they taught
## Note: a student enrolled in 2 courses will be counted as 2 for a instrcutor, as there is no way to distinguish them!
instructors = set([course['instructor'] for course in courses])
subTuples = [(i, sum(filterItemByGroup(i, "enrolled", group = "instructor"))) for i in instructors]
topNFromTuple(subTuples, 20)

In [ ]:
## h: list the top 20 courses in terms of number of students taking that course (where you combine different sections and semesters, i.e. just use the subject and course number)
## Note: here I just use the subject and course number!!
codes = set([course['code'] for course in courses])
subTuples = [(i, sum(filterItemByGroup(i, "enrolled", group = "code"))) for i in codes]
topNFromTuple(subTuples, 20)

In [ ]:
# i: Create your own interesting question (each team member creates their own) and use Python to answer that question.
# zihao: who is the instructor involved in teaching courses in most subjects?
import pandas as pd
subdata = [[i['instructor'], i['subject']] for i in courses]
df = pd.DataFrame(subdata, columns = ['instructor', 'subject'])

df_filter = df.groupby('instructor')['subject'].apply(lambda x: len(set(x)))
df_filter.sort_values(ascending = False)

df[df['instructor'] == df_filter.sort_values(ascending = False).index[1]].drop_duplicates()